Target: Reduce Dropout Rate , optimize train transforms through data augmentation and change optimizer

Results:

Parameters: 5710

Best Train Accuracy: 98.12


Best Test Accuracy: 99.36

Analysis: better convergence but still accuracy below 99.4%

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
from tqdm import tqdm
from torchsummary import summary

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # Initial Block: 28x28x1 -> 26x26x10
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=3, padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.Dropout(0.005)  # Slightly increased dropout
        )

        # Conv Block 1: 26x26x10 -> 24x24x16
        self.conv2 = nn.Sequential(
            nn.Conv2d(10, 16, kernel_size=3, padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.005)
        )

        # Transition Block: 24x24x16 -> 12x12x10
        self.trans1 = nn.Sequential(
            nn.Conv2d(16, 10, kernel_size=1, bias=False),
            nn.MaxPool2d(2, 2)
        )

        # Conv Block 2: 12x12x10 -> 10x10x16
        self.conv3 = nn.Sequential(
            nn.Conv2d(10, 16, kernel_size=3, padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.005)
        )

        # Conv Block 3: 10x10x16 -> 8x8x16
        self.conv4 = nn.Sequential(
            nn.Conv2d(16, 16, kernel_size=3, padding=0, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.005)
        )

        self.gap = nn.Sequential(
            nn.AvgPool2d(kernel_size=8)
        )

        self.conv5 = nn.Sequential(
            nn.Conv2d(16, 10, kernel_size=1, padding=0, bias=False)
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.trans1(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.gap(x)
        x = self.conv5(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)

def train(model, device, train_loader, optimizer, scheduler, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        y_pred = model(data)
        loss = F.nll_loss(y_pred, target)

        loss.backward()
        optimizer.step()
        scheduler.step()

        pred = y_pred.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)

        pbar.set_description(desc=f'Epoch={epoch} Loss={loss.item():.4f} Batch={batch_idx} Accuracy={100*correct/processed:.2f}')
    return 100 * correct / processed

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100.*correct/len(test_loader.dataset):.2f}%)\n')
    return 100. * correct / len(test_loader.dataset)

def main():
    torch.manual_seed(1)
    batch_size = 128
    epochs = 15

    # Enhanced data augmentation with more aggressive transformations

    train_transform = transforms.Compose([
        transforms.RandomRotation((-12.0, 12.0)),  # Increased rotation
        transforms.RandomAffine(
            degrees=0,
            translate=(0.13, 0.13),  # Increased translation
            scale=(0.82, 1.18),      # Increased scale range
            shear=(-3, 3)            # Added shear
        ),
        transforms.ColorJitter(
            brightness=0.15,          # Increased brightness adjustment
            contrast=0.15            # Increased contrast adjustment
        ),
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)),
        transforms.RandomErasing(p=0.2, scale=(0.02, 0.1))  # Added random erasing
    ])

    test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    train_loader = DataLoader(
        datasets.MNIST('../data', train=True, download=True, transform=train_transform),
        batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True
    )

    test_loader = DataLoader(
        datasets.MNIST('../data', train=False, transform=test_transform),
        batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True
    )


    from torch.optim.lr_scheduler import StepLR

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = Net().to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-5)

    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=0.015,
        epochs=epochs,
        steps_per_epoch=len(train_loader),
        pct_start=0.15,
        div_factor=15,
        final_div_factor=100
    )

    summary(model, input_size=(1, 28, 28))

    accuracies_above_99_4 = 0
    best_acc = 0
    for epoch in range(epochs):
        train_acc = train(model, device, train_loader, optimizer, scheduler, epoch)
        test_acc = test(model, device, test_loader)
        if test_acc >= 99.4:
            accuracies_above_99_4 += 1
        if test_acc > best_acc:
            best_acc = test_acc
            torch.save(model.state_dict(), 'best_model.pth')

        print(f"Times achieved ≥99.4%: {accuracies_above_99_4}")

if __name__ == '__main__':
    main()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]              90
              ReLU-2           [-1, 10, 26, 26]               0
       BatchNorm2d-3           [-1, 10, 26, 26]              20
           Dropout-4           [-1, 10, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           1,440
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
           Dropout-8           [-1, 16, 24, 24]               0
            Conv2d-9           [-1, 10, 24, 24]             160
        MaxPool2d-10           [-1, 10, 12, 12]               0
           Conv2d-11           [-1, 16, 10, 10]           1,440
             ReLU-12           [-1, 16, 10, 10]               0
      BatchNorm2d-13           [-1, 16, 10, 10]              32
          Dropout-14           [-1, 16,

Epoch=0 Loss=0.3178 Batch=468 Accuracy=70.76: 100%|██████████| 469/469 [00:44<00:00, 10.57it/s]



Test set: Average loss: 0.1546, Accuracy: 9563/10000 (95.63%)

Times achieved ≥99.4%: 0


Epoch=1 Loss=0.1458 Batch=468 Accuracy=93.32: 100%|██████████| 469/469 [00:43<00:00, 10.80it/s]



Test set: Average loss: 0.0799, Accuracy: 9751/10000 (97.51%)

Times achieved ≥99.4%: 0


...


Test set: Average loss: 0.0180, Accuracy: 9936/10000 (99.36%)

Times achieved ≥99.4%: 0


Epoch=13 Loss=0.0400 Batch=468 Accuracy=98.12: 100%|██████████| 469/469 [00:43<00:00, 10.72it/s]



Test set: Average loss: 0.0172, Accuracy: 9934/10000 (99.34%)

Times achieved ≥99.4%: 0


Epoch=14 Loss=0.0977 Batch=468 Accuracy=98.08: 100%|██████████| 469/469 [00:43<00:00, 10.77it/s]



Test set: Average loss: 0.0172, Accuracy: 9932/10000 (99.32%)

Times achieved ≥99.4%: 0
